In [1]:
import glob
import pandas as pd
import time
import re
from outlook_msg import Message

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
%%time
conf = SparkConf()
conf.set("spark.network.timeout", "1000000")
conf.set("spark.executor.heartbeatInterval", "1000000")
conf.set("spark.speculation","false")
conf.set("spark.app.name","blahered")
spark = SparkSession.builder.config(conf = conf).master("local[*]").enableHiveSupport().getOrCreate() #("local[*]") uses all the cpu cores#

Wall time: 6.21 s


In [5]:
spark

In [6]:
sc = spark.sparkContext

In [7]:
from outlook_msg import Message

with open(r'C:\Users\QuadAdmin\Downloads\files\msg\001.msg', "r", encoding="utf-8") as msg_file:
    msg = Message(msg_file)
msg.body

: Mon-Fri 9-18 GMT+8 \r\n\r\n\r\n\r\n \r\n\r\nFrom: Raghu Ram Mahankali (Quadrant Resource) <raghurama@quadrantresource.com> \r\nSent: Wednesday, April 14, 2021 10:52 PM\r\nTo: Yu Huang <yuhuang1@microsoftsupport.com>\r\nCc: rceop@microsoft.com\r\nSubject: RE: Help needed to execute python commands on jupyt... - TrackingID#2104120060004581\r\n\r\n \r\n\r\nAlso, I observed that any other function(to be precise, any generator) like glob/os.walk/Path/rglob that is trying to traverse the folders is getting into an infinite loop.\r\n\r\n \r\n\r\nThanks,\r\n\r\nRaghu Ram Mahankali.\r\n\r\n \r\n\r\nFrom: Raghu Ram Mahankali (Quadrant Resource) <mailto:raghurama@quadrantresource.com> \r\nSent: Wednesday, April 14, 2021 20:19\r\nTo: Yu Huang <mailto:yuhuang1@microsoftsupport.com> \r\nCc: rceop@microsoft.com <mailto:rceop@microsoft.com> \r\nSubject: RE: Help needed to execute python commands on jupyt... - TrackingID#2104120060004581\r\n\r\n \r\n\r\nHello Yu Huang,\r\n\r\n \r\n\r\nThanks for the 

In [8]:
def readmsgFiles(file):
    with open(file,"r", encoding="utf-8") as msg_file:
        msg = Message(msg_file)
        msg_body = msg.body
    return(re.sub('\s+'," ",msg_body).strip())

In [9]:
readmsgFiles(r'C:\Users\QuadAdmin\Downloads\files\msg\001.msg')

'Hi Raghu, Thanks for your response. I am glad to hear os.listdir is working now. For the infinite loop issue, as mentioned in the previous email, if there are too many files while using the function (glob, path), it will take long time. Sometimes it may be failed with no error (stuck in infinite loop). Do you have a chance to look at how many files you were handling? I tried to login to your cluster to run the command to check further. But it seems that the cluster is deleted. May I ask if you have other action plan? If you have question or concern within the scope, please feel free to let me know. Best Regards, Yu Huang | Support Engineer | APAC Big Data Support Team Tel. +86 (21) 52638969 | Email: yuhuang1@microsoft.com | Working Hours: Mon-Fri 9-18 GMT+8 From: Raghu Ram Mahankali (Quadrant Resource) <raghurama@quadrantresource.com> Sent: Wednesday, April 14, 2021 10:52 PM To: Yu Huang <yuhuang1@microsoftsupport.com> Cc: rceop@microsoft.com Subject: RE: Help needed to execute python

In [10]:
%%time
DF = pd.DataFrame(columns=["fileName", "text"])
st_time = time.time()
paths = glob.glob(r'C:\Users\QuadAdmin\Downloads\files\**\*.msg',recursive = True)

Wall time: 623 ms


In [11]:
len(paths)

73764

In [12]:
DF["fileName"] = paths

In [13]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\files\msg\001.msg,NaN
1,C:\Users\QuadAdmin\Downloads\files\msg\1.msg,NaN
2,C:\Users\QuadAdmin\Downloads\files\msg\10.msg,NaN
3,C:\Users\QuadAdmin\Downloads\files\msg\100.msg,NaN
4,C:\Users\QuadAdmin\Downloads\files\msg\1000.msg,NaN
...,...,...
73759,C:\Users\QuadAdmin\Downloads\files\msg\9995.msg,NaN
73760,C:\Users\QuadAdmin\Downloads\files\msg\9996.msg,NaN
73761,C:\Users\QuadAdmin\Downloads\files\msg\9997.msg,NaN
73762,C:\Users\QuadAdmin\Downloads\files\msg\9998.msg,NaN


In [14]:
readmsgFiles(DF["fileName"][0])

'Hi Raghu, Thanks for your response. I am glad to hear os.listdir is working now. For the infinite loop issue, as mentioned in the previous email, if there are too many files while using the function (glob, path), it will take long time. Sometimes it may be failed with no error (stuck in infinite loop). Do you have a chance to look at how many files you were handling? I tried to login to your cluster to run the command to check further. But it seems that the cluster is deleted. May I ask if you have other action plan? If you have question or concern within the scope, please feel free to let me know. Best Regards, Yu Huang | Support Engineer | APAC Big Data Support Team Tel. +86 (21) 52638969 | Email: yuhuang1@microsoft.com | Working Hours: Mon-Fri 9-18 GMT+8 From: Raghu Ram Mahankali (Quadrant Resource) <raghurama@quadrantresource.com> Sent: Wednesday, April 14, 2021 10:52 PM To: Yu Huang <yuhuang1@microsoftsupport.com> Cc: rceop@microsoft.com Subject: RE: Help needed to execute python

In [15]:
readmsgFiles_udf = udf(readmsgFiles, StringType())

In [16]:
pathsDF = spark.createDataFrame(DF)

In [17]:
pathsDF.show(5)

+--------------------+----+
|            fileName|text|
+--------------------+----+
|C:\Users\QuadAdmi...| NaN|
|C:\Users\QuadAdmi...| NaN|
|C:\Users\QuadAdmi...| NaN|
|C:\Users\QuadAdmi...| NaN|
|C:\Users\QuadAdmi...| NaN|
+--------------------+----+
only showing top 5 rows



In [18]:
%%time
pathsDF.select("fileName", readmsgFiles_udf("fileName").alias("text")).show(5)

+--------------------+--------------------+
|            fileName|                text|
+--------------------+--------------------+
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
+--------------------+--------------------+
only showing top 5 rows

Wall time: 3.15 s


In [23]:
%%time
pathsDF.select("fileName", readmsgFiles_udf("fileName").alias("text")).write.mode("append").saveAsTable("filesMsg_93gig",path="D:\\tables")

Wall time: 11min 23s
